In [ ]:
import baostock as bs
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# sh.601328 sh.601939 sh.601988

lg = bs.login()
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctCh
rs = bs.query_history_k_data_plus(
    code="sh.",# 股票代码 sz.xxxxxx 或者 sh.xxxxxx
    # fields="date,time,open,high,low,close,volume,adjustflag", # data: YYYY-MM-DD格式
    fields="date,code,open,high,low,close,volume,amount,adjustflag,turn",
    # time: 20190102093500000, open: 开盘价, high: 最高价, low: 最低价, close: 收盘价, volume: 量(股数 1手 = 100股)
    start_date="2026-01-01",# YYYY-MM-DD格式
    end_date="2026-01-20",
    frequency="d",# d for day, 5 for 5 minutes and 15 30 60 so on, w for week, m for month
    adjustflag="1",# 1: 后复权  2: 前复权  3: 不复权  模型后复权通常更常用
)
# rs 的成员函数 rs.error_code	错误码（是否成功）
# rs.error_msg	错误描述信息
# rs.fields	字段列表（你传入的 fields）
# rs.get_row_data()	获取当前一行数据（字符串列表）
# rs.next()	移动到下一条记录，用于遍历结果
print(rs.error_msg)
print("-")
print(rs.next())
a = rs.get_row_data()
print(a)

print(rs.fields)
bs.logout()



In [ ]:
import baostock as bs
import pandas as pd
from tqdm import tqdm

# -------------------------------
# 目标设置
# -------------------------------
code = "sh.601328"           # Baostock 股票代码
freq = "5"                  # 5 分钟
start_date = "2019-01-01"   # 五年前起始日期
end_date = pd.to_datetime("today").strftime("%Y-%m-%d")  # 当前日期

output_csv = f"{code}.csv"

# -------------------------------
# 登录 Baostock
# -------------------------------
lg = bs.login()
print("登录状态:", lg.error_code, lg.error_msg)

# -------------------------------
# 拉取函数
# -------------------------------
def fetch_baostock_5min(code, start_date, end_date):
    all_data = []

    # 生成日期列表用于进度
    date_list = pd.date_range(start=start_date, end=end_date, freq="D")

    for dt in tqdm(date_list, desc="拉取日期进度"):
        day_str = dt.strftime("%Y-%m-%d")

        rs = bs.query_history_k_data_plus(
            code,
            "date,time,open,high,low,close,volume",
            start_date=day_str,
            end_date=day_str,
            frequency=freq,
            adjustflag="1"
        )

        # 读取数据
        data_list = []
        while rs.error_code == "0" and rs.next():
            data_list.append(rs.get_row_data())

        if data_list:
            df_day = pd.DataFrame(data_list, columns=rs.fields)
            all_data.append(df_day)

    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        return pd.DataFrame()

# -------------------------------
# 执行拉取
# -------------------------------
print(f"开始从 {start_date} 到 {end_date} 拉取 {code} 的 5 分钟数据...")
df_5min = fetch_baostock_5min(code, start_date, end_date)

# -------------------------------
# 保存 CSV
# -------------------------------
if not df_5min.empty:
    df_5min.to_csv(output_csv, index=False)
    print("保存成功:", output_csv)
    print("数据样例:\n", df_5min.head())
    print("总行数:", len(df_5min))
else:
    print("没有拉取到任何数据！")

bs.logout()


login success!
登录状态: 0 success
开始从 2019-01-01 到 2026-01-14 拉取 sh.601328 的 5 分钟数据...


拉取日期进度: 100%|██████████| 2571/2571 [07:21<00:00,  5.82it/s]


保存成功: fsh.601328.csv
数据样例:
          date               time           open           high            low  \
0  2019-01-02  20190102093500000  10.1297139100  10.1823814000  10.1121580800   
1  2019-01-02  20190102094000000  10.1297139100  10.1472697400  10.0946022500   
2  2019-01-02  20190102094500000  10.1121580800  10.1121580800  10.0594905900   
3  2019-01-02  20190102095000000  10.0594905900  10.0594905900  10.0068231000   
4  2019-01-02  20190102095500000  10.0243789300  10.0243789300   9.9717114400   

           close   volume  
0  10.1297139100  2513490  
1  10.0946022500  1420900  
2  10.0594905900  1725200  
3  10.0068231000  4243200  
4   9.9717114400  2477100  
总行数: 81888
logout success!
